In [ ]:
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

df = pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [5]:
X = df.drop('Outcome', axis=1)
y = df['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42,n_jobs=-1
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [8]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy

study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2025-12-07 16:57:24,958] A new study created in memory with name: no-name-65e7351f-0add-4233-a7ba-ac8fe4a0c06a
[I 2025-12-07 16:57:26,020] Trial 0 finished with value: 0.7765363128491621 and parameters: {'n_estimators': 74, 'max_depth': 19}. Best is trial 0 with value: 0.7765363128491621.
[I 2025-12-07 16:57:27,534] Trial 1 finished with value: 0.7765363128491619 and parameters: {'n_estimators': 125, 'max_depth': 12}. Best is trial 0 with value: 0.7765363128491621.
[I 2025-12-07 16:57:28,901] Trial 2 finished with value: 0.7783985102420856 and parameters: {'n_estimators': 114, 'max_depth': 19}. Best is trial 2 with value: 0.7783985102420856.
[I 2025-12-07 16:57:30,906] Trial 3 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 177, 'max_depth': 9}. Best is trial 2 with value: 0.7783985102420856.
[I 2025-12-07 16:57:32,744] Trial 4 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 159, 'max_depth': 16}. Best is trial 2 with value: 0.778398

In [9]:
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 121, 'max_depth': 18}


In [10]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)

test_accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


## Samplers in Optuna

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42,n_jobs=-1
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [12]:
# Implement RandomSearch for hyperparameter optimization

study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-12-07 16:59:46,730] A new study created in memory with name: no-name-fdd666f1-f434-4d63-8c3e-6840eefe819f
[I 2025-12-07 16:59:47,814] Trial 0 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 84, 'max_depth': 8}. Best is trial 0 with value: 0.7672253258845437.
[I 2025-12-07 16:59:49,357] Trial 1 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 124, 'max_depth': 10}. Best is trial 0 with value: 0.7672253258845437.
[I 2025-12-07 16:59:51,231] Trial 2 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 159, 'max_depth': 16}. Best is trial 2 with value: 0.7746741154562384.
[I 2025-12-07 16:59:51,992] Trial 3 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 58, 'max_depth': 13}. Best is trial 2 with value: 0.7746741154562384.
[I 2025-12-07 16:59:54,499] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 197, 'max_depth': 12}. Best is trial 2 with value: 0.7746741

In [13]:
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7821229050279329
Best hyperparameters: {'n_estimators': 117, 'max_depth': 15}


In [14]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)

test_accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


In [16]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [17]:
# Implement GridSearch for hyperparameter optimization

study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2025-12-07 17:04:12,341] A new study created in memory with name: no-name-2d3d5056-14c1-4e09-90be-59449accef78
[I 2025-12-07 17:04:13,517] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-12-07 17:04:15,366] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-12-07 17:04:16,357] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-12-07 17:04:17,434] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-12-07 17:04:18,037] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [18]:
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [19]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)

test_accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


## Optuna Visualizations

In [20]:
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [21]:
# 1. Optimization History
plot_optimization_history(study).show()

In [22]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [23]:
# 3. Slice Plot
plot_slice(study).show()

In [24]:
# 4. Contour Plot
plot_contour(study).show()

In [25]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

## Optimizing Multiple ML Models

In [26]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [27]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [28]:
# Create a study and optimize it
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-12-07 17:10:40,290] A new study created in memory with name: no-name-c7f8a8b1-d51c-4917-be33-8f61c3c4fba9
[I 2025-12-07 17:10:41,325] Trial 0 finished with value: 0.7579143389199254 and parameters: {'classifier': 'RandomForest', 'n_estimators': 219, 'max_depth': 18, 'min_samples_split': 9, 'min_samples_leaf': 6, 'bootstrap': True}. Best is trial 0 with value: 0.7579143389199254.
[I 2025-12-07 17:10:41,865] Trial 1 finished with value: 0.756052141527002 and parameters: {'classifier': 'RandomForest', 'n_estimators': 129, 'max_depth': 4, 'min_samples_split': 9, 'min_samples_leaf': 8, 'bootstrap': True}. Best is trial 0 with value: 0.7579143389199254.
[I 2025-12-07 17:10:41,904] Trial 2 finished with value: 0.7635009310986964 and parameters: {'classifier': 'SVM', 'C': 0.26914677557195893, 'kernel': 'sigmoid', 'gamma': 'scale'}. Best is trial 2 with value: 0.7635009310986964.
[I 2025-12-07 17:10:42,434] Trial 3 finished with value: 0.7541899441340782 and parameters: {'classifier': '

In [29]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.14817301407047181, 'kernel': 'linear', 'gamma': 'auto'}
Best trial accuracy: 0.7895716945996275


In [30]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.757914,2025-12-07 17:10:40.291426,2025-12-07 17:10:41.325654,0 days 00:00:01.034228,NaN,True,RandomForest,NaN,NaN,NaN,18.0,6.0,9.0,219.0,COMPLETE
1,1,0.756052,2025-12-07 17:10:41.325654,2025-12-07 17:10:41.865750,0 days 00:00:00.540096,NaN,True,RandomForest,NaN,NaN,NaN,4.0,8.0,9.0,129.0,COMPLETE
2,2,0.763501,2025-12-07 17:10:41.866768,2025-12-07 17:10:41.904775,0 days 00:00:00.038007,0.269147,NaN,SVM,scale,sigmoid,NaN,NaN,NaN,NaN,NaN,COMPLETE
3,3,0.754190,2025-12-07 17:10:41.904775,2025-12-07 17:10:42.434963,0 days 00:00:00.530188,NaN,True,RandomForest,NaN,NaN,NaN,3.0,8.0,4.0,126.0,COMPLETE
4,4,0.770950,2025-12-07 17:10:42.436511,2025-12-07 17:10:42.460220,0 days 00:00:00.023709,2.203949,NaN,SVM,scale,rbf,NaN,NaN,NaN,NaN,NaN,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.703911,2025-12-07 17:11:09.495255,2025-12-07 17:11:09.522196,0 days 00:00:00.026941,0.100252,NaN,SVM,auto,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.785847,2025-12-07 17:11:09.523290,2025-12-07 17:11:09.549928,0 days 00:00:00.026638,0.304490,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.785847,2025-12-07 17:11:09.549928,2025-12-07 17:11:09.576331,0 days 00:00:00.026403,0.195073,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.785847,2025-12-07 17:11:09.577340,2025-12-07 17:11:09.607445,0 days 00:00:00.030105,0.235041,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [31]:
study.trials_dataframe()['params_classifier'].value_counts()

params_classifier
SVM                 78
RandomForest        13
GradientBoosting     9
Name: count, dtype: int64

In [32]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

params_classifier
GradientBoosting    0.744672
RandomForest        0.764360
SVM                 0.774961
Name: value, dtype: float64

In [33]:
# 1. Optimization History
plot_optimization_history(study).show()

In [34]:
# 3. Slice Plot
plot_slice(study).show()

In [35]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [38]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mlogloss")  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        evals=[(dtrain, "train"), (dtest, "eval")],  # Ensure the eval datasets and names are specified
        early_stopping_rounds=30,
        callbacks=[pruning_callback]    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")


[I 2025-12-07 17:17:11,546] A new study created in memory with name: no-name-02f9d625-e160-4444-8e7e-d5627603b060


[0]	train-mlogloss:0.79597	eval-mlogloss:0.77841
[1]	train-mlogloss:0.70578	eval-mlogloss:0.67230
[2]	train-mlogloss:0.61132	eval-mlogloss:0.56561
[3]	train-mlogloss:0.48449	eval-mlogloss:0.43043
[4]	train-mlogloss:0.41556	eval-mlogloss:0.36020
[5]	train-mlogloss:0.36101	eval-mlogloss:0.30099
[6]	train-mlogloss:0.32187	eval-mlogloss:0.25888
[7]	train-mlogloss:0.32184	eval-mlogloss:0.25881
[8]	train-mlogloss:0.32176	eval-mlogloss:0.25861
[9]	train-mlogloss:0.31993	eval-mlogloss:0.25350
[10]	train-mlogloss:0.30938	eval-mlogloss:0.24174
[11]	train-mlogloss:0.30887	eval-mlogloss:0.24258
[12]	train-mlogloss:0.30749	eval-mlogloss:0.24148
[13]	train-mlogloss:0.30698	eval-mlogloss:0.24122
[14]	train-mlogloss:0.30609	eval-mlogloss:0.24002
[15]	train-mlogloss:0.30600	eval-mlogloss:0.23940
[16]	train-mlogloss:0.30607	eval-mlogloss:0.23973
[17]	train-mlogloss:0.30590	eval-mlogloss:0.23997
[18]	train-mlogloss:0.30611	eval-mlogloss:0.23961
[19]	train-mlogloss:0.30538	eval-mlogloss:0.23927
[20]	train

[I 2025-12-07 17:17:12,057] Trial 0 finished with value: 1.0 and parameters: {'lambda': 1.356169069707104e-05, 'alpha': 0.04057651453652077, 'eta': 0.2622108107582189, 'gamma': 0.007031239930800896, 'max_depth': 8, 'min_child_weight': 10, 'subsample': 0.6774152075501074, 'colsample_bytree': 0.5744636996039979}. Best is trial 0 with value: 1.0.


[0]	train-mlogloss:0.89452	eval-mlogloss:0.88921
[1]	train-mlogloss:0.74565	eval-mlogloss:0.72852
[2]	train-mlogloss:0.62605	eval-mlogloss:0.60423
[3]	train-mlogloss:0.53179	eval-mlogloss:0.50836
[4]	train-mlogloss:0.45828	eval-mlogloss:0.43249
[5]	train-mlogloss:0.39542	eval-mlogloss:0.36188
[6]	train-mlogloss:0.34681	eval-mlogloss:0.30777
[7]	train-mlogloss:0.30661	eval-mlogloss:0.26255
[8]	train-mlogloss:0.27448	eval-mlogloss:0.22574
[9]	train-mlogloss:0.24865	eval-mlogloss:0.19867
[10]	train-mlogloss:0.22767	eval-mlogloss:0.17426
[11]	train-mlogloss:0.21137	eval-mlogloss:0.15482
[12]	train-mlogloss:0.19650	eval-mlogloss:0.13922
[13]	train-mlogloss:0.19122	eval-mlogloss:0.13148
[14]	train-mlogloss:0.18105	eval-mlogloss:0.12013
[15]	train-mlogloss:0.17832	eval-mlogloss:0.11622
[16]	train-mlogloss:0.17503	eval-mlogloss:0.11129
[17]	train-mlogloss:0.17124	eval-mlogloss:0.10749
[18]	train-mlogloss:0.16969	eval-mlogloss:0.10798
[19]	train-mlogloss:0.16648	eval-mlogloss:0.10378
[20]	train

[I 2025-12-07 17:17:12,722] Trial 1 finished with value: 1.0 and parameters: {'lambda': 1.162476427186061e-08, 'alpha': 1.973960511243152e-06, 'eta': 0.16474356755030986, 'gamma': 1.4390481942019687e-07, 'max_depth': 3, 'min_child_weight': 7, 'subsample': 0.887682855517182, 'colsample_bytree': 0.8635727418794062}. Best is trial 0 with value: 1.0.


...

[0]	train-mlogloss:1.02920	eval-mlogloss:1.03035
[1]	train-mlogloss:0.98410	eval-mlogloss:0.99432
[2]	train-mlogloss:0.92665	eval-mlogloss:0.93578
[3]	train-mlogloss:0.86982	eval-mlogloss:0.87603
[4]	train-mlogloss:0.81925	eval-mlogloss:0.82307


[I 2025-12-07 17:17:16,492] Trial 48 pruned. Trial was pruned at iteration 4.


[0]	train-mlogloss:0.78141	eval-mlogloss:0.76712


[I 2025-12-07 17:17:16,520] Trial 49 pruned. Trial was pruned at iteration 1.


Best trial: {'lambda': 1.356169069707104e-05, 'alpha': 0.04057651453652077, 'eta': 0.2622108107582189, 'gamma': 0.007031239930800896, 'max_depth': 8, 'min_child_weight': 10, 'subsample': 0.6774152075501074, 'colsample_bytree': 0.5744636996039979}
Best accuracy: 1.0


In [39]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()